In [ ]:
'''
GAMMA-ITERATIVE PROCESS (CVXPY) 2
'''
# Max number of arrays
n = N(P,H)

# Piecewise linear circle approximation
angles = np.linspace(0,np.pi/2,K)
L = np.block([[np.zeros(2)], [np.cos(angles[:,None]), np.sin(angles[:,None])]])
Al, bl, ll, lu = vert2con(L)
non_ort_indices = np.argsort(bl)[0,2:]
Al, bl = Al[non_ort_indices], bl[:,non_ort_indices]
ll, lu = ll[non_ort_indices], lu[non_ort_indices]

# big-M
bigMm = diameter(P) // W
bigMu = np.sum(Al,1,keepdims=True)
bigMd = H*(np.cos(np.pi/4) + np.sin(np.pi/4)/np.tan(np.min(sun_elev))) + D_OFF

# Matrices 
O = np.vstack([np.zeros(n), np.tril(np.ones([n,n]))])
Ao, bo, _, _ = vert2con(O)
Ah, bh, _, _ = vert2con(P)

# Variables
x = cp.Variable((n,4), name="x")
y = cp.Variable((n,4), name="y")
m = cp.Variable((n,1), nonneg=True, name="m")
z = cp.Variable((n,1), boolean=True, name="z")
w = cp.Variable((K-1,1), boolean=True, name="w")
u = cp.Variable((2,1), nonneg=True, name="u") # sin(tilt), cos(tilt)
v = cp.Parameter(2, value=[0,-1], name="v") # but I consider it as a variable

# Constants
constants = {
    "sun_azmth": sun_azmth,
    "sun_elev": sun_elev,
    "dni": dni,
    "dhi": dhi,
    "ghi": ghi,
    "alb": ALB,
    "H": H,
    "W": W,
    "d_off": D_OFF
    "CR": CR,
    "CP": CP
}

# Expressions
def csge(**var, **const):
    v, sun_azmth = var["v"], const["sun_azmth"]
    return cp.multiply(np.cos(sun_azmth),v[1]) + cp.multiply(np.sin(sun_azmth),v[0])

def shad(**var, **const):
    u, H, sun_elev = var["u"], const["H"], const["sun_elev"]
    return H*(u[1,0] + u[0,0]*csge(var,const)/np.tan(sun_elev))

def dist(**var, **const):
    v, x, y = var["v"], var["x"], var["y"]
    return v[0]*(x[:-1,0] - x[1:,0]) + v[1]*(y[:-1,0] - y[1:,0])

def paoi(**var, **const):
    u, sun_elev = var["u"], const["sun_elev"]
    return cp.multiply(u[1,0], np.sin(sun_elev)) + cp.multiply(u[0,0], cp.multiply(np.cos(sun_elev),csge(var,const)))
                                                               
def beam(**var, **const):
    dhi = const["dhi"]
    return cp.multiply(dni, paoi(var,const))
                                                               
def diff(**var, **const):
    u, dhi = var["u"], const["dhi"]                                                                  
    return 0.5*dhi*(1 + u[1,0])  
                                                               
def grnd(**var, **const):
    u, alb, ghi = var["u"], const["alb"], const["ghi"]    
    return 0.5*alb*ghi*(1 - u[1,0]) 

def area(**var, **const):
    m, W, H = var["m"], const["W"], const["H"]
    return W*H*cp.sum(m)

def irrd(**var, **const):
    return beam(var, const) + diff(var, const) + grnd(var, const)

def treq(**var):
    u = var["u"]
    return cp.sum_squares(u)

def cost(**var, **const):
    z, CP, CR = var["z"], const["CP"], const["CR"]
    return area(var,const)*CP + cp.sum(z)*CR

def power(**var, **const):
    return area(var, const)*irrd(var, const)

def log_power(**var, **const):
    return cp.log(area(var, const)) + cp.log(irrd(var, const))
                                                               
# csge = cp.multiply(np.cos(sun_azmth),v[1]) + cp.multiply(np.sin(sun_azmth),v[0])
# shad = H*(u[1,0] + u[0,0]*csge(v,sun_azmth)/np.tan(sun_elev))
# #shad = H*(u[1,0] + u[0,0]/np.tan(np.min(sun_elev)))
# dist = v[0]*(x[:-1,0] - x[1:,0]) + v[1]*(y[:-1,0] - y[1:,0])
# paoi = cp.multiply(u[1,0], np.sin(sun_elev)) + cp.multiply(u[0,0], cp.multiply(np.cos(sun_elev),csge))
# beam = cp.multiply(dni, paoi)
# diff = 0.5*dhi*(1 + u[1,0])
# grnd = 0.5*ALB*ghi*(1 - u[1,0]) 
# area = W*H*cp.sum(m)
# irrd = beam + diff + grnd
# treq = cp.sum_squares(u)
# cost = area*CP + cp.sum(z)*CR

# power         = cp.multiply(area, irrd)
# log_power     = cp.log(area) + cp.log(irrd)
# min_log_power = cp.min(log_power)

# Constraints
constraints = [
    # Array coordinates 
    x[:,1] == x[:,0] - v[1]*W*m[:,0], y[:,1] == y[:,0] + v[0]*W*m[:,0],
    x[:,2] == x[:,1] - v[0]*H*u[1,0], y[:,2] == y[:,1] - v[1]*H*u[1,0],
    x[:,3] == x[:,0] - v[0]*H*u[1,0], y[:,3] == y[:,0] - v[1]*H*u[1,0],
    # Array ordering
    Ao @ z <= bo.T,
    # Trigonometric approximation 
    Al @ u <= bl.T + cp.multiply((1 - w),bigMu), 
    Al @ u >= cp.multiply(bl.T,w),
    u.T >= cp.sum(cp.multiply(ll, w),0,keepdims=True), 
    u.T <= cp.sum(cp.multiply(lu, w),0,keepdims=True),
    cp.sum(w) == 1,
    # z-m logic constraints
    m <= bigMm*z, 
    m >= z,
    # Arrays' inside polygon
    Ah @ cp.vstack([cp.vec(x), cp.vec(y)]) <= np.kron(np.ones(4*n), bh.T),
]
_dist = dist(x=x,y=y,v=v)
for i in range(n - 1):
    constraints += [_dist[i] >= cp.max(shad(u=u,constants)) + constants["d_off"] - bigMd*(1-z[i+1,0])]
        
# Problem type
if PROBLEM_TYPE == "Power":
    objective = cp.Maximize(cp.min(log_power()))
elif PROBLEM_TYPE == "Area":
    objective = cp.Minimize(cost)
    constraints += [log_power >= np.log(power_sim)] # Power requirements

# Compile 
prob = cp.Problem(objective, constraints)

# Iterative process
for g in np.linspace(np.pi/2, 3*np.pi/2, S):
    
    v.value = [np.sin(g), np.cos(g)]
    results = prob.solve(solver=cp.MOSEK, verbose=True)
    
    # Plotting 
    plot_layout(x.value, y.value, z.value, P)

                                                               

In [ ]:
# GAMMA-ITERATIVE PROCESS (CVXPY)
optim = dict()
optim["prob"]  = None
optim["azmth"] = None
optim["value"] = np.inf if PROB_TYPE == "Area" else -np.inf
        
elapsed_time = list()

# Piecewise linear circle approximation
angles = np.linspace(0,np.pi/2,K)
L = np.block([[np.zeros(2)], [np.cos(angles[:,None]), np.sin(angles[:,None])]])
Al, bl, ll, lu = vert2con(L)
non_ort_indices = np.argsort(bl)[0,2:]
Al, bl = Al[non_ort_indices], bl[:,non_ort_indices]
ll, lu = ll[non_ort_indices], lu[non_ort_indices]

# Iterative process
arr_azmths = np.linspace(np.pi/2, 3*np.pi/2, S)
for i,curr_arr_azmth in enumerate(arr_azmths):
    rot_poly = poly @ rotM(curr_arr_azmth).T
    
    N_curr = int((np.max(rot_poly[:,1]) - np.min(rot_poly[:,1])) // H)
    N = min(N_max, N_curr)
    
    time_start = time.time()
    
    # Create the varaibles
    x = cp.Variable((1,N), name="x")
    y = cp.Variable((1,N), name="y")
    m = cp.Variable((1,N), integer=True, name="m")
    z = cp.Variable((1,N), boolean=True, name="z")
    w = cp.Variable((1,K-1), boolean=True, name="w")
    u = cp.Variable((1,2), nonneg=True, name="u") # sin(tilt), cos(tilt)

    r = cp.bmat([[x, x + m*W, x + m*W, x], [y, y, y + H, y + H]])

    # Arrays ordering 
    O = np.block([
        [np.zeros(N)],
        [np.tril(np.ones([N,N]))]])
    Ao, bo, _, _ = vert2con(O)

    # Hyperplane (values of Ah and bh are updated iteratively)
    Ah, bh, _, _ = vert2con(rot_poly)
    bh = np.kron(np.ones(4*N), bh.T)
    zh = cp.kron(np.ones([V,4]), z)

    # Trigonometric equality
    treq = cp.sum_squares(u)

    # Distance between arrays
    d = H*(u[0,1] + u[0,0]*np.cos(curr_arr_azmth - sun_azmth)/np.tan(sun_elev))

    # Irradiance
    paoi = cp.multiply(u[0,1], np.sin(sun_elev)) + \
           cp.multiply(u[0,0], np.cos(sun_elev)*np.cos(sun_azmth - curr_arr_azmth))
    beam = cp.multiply(irr["beam"], paoi)
    diff = 0.5*irr["diff"]*(1 + u[0,1])
    grnd = 0.5*ALB*irr["grnd"]*(1 - u[0,1]) 

    # Total area, irradiance and power
    area  = W * H * cp.sum(m)
    irrd  = 1e-3 * (beam + diff + grnd)
    power = cp.multiply(area, irrd)

    # Constraints 
    ltrig = [Al @ u.T <= bl.T + (1 - w.T)*bigM, 
             Al @ u.T >= bl.T - (1 - w.T)*bigM,
             u >= np.ones([1,K-1]) @ cp.multiply(ll, w.T), 
             u <= np.ones([1,K-1]) @ cp.multiply(lu, w.T),
             cp.sum(w) == 1]
    dist = [y[0,i+1] - y[0,i] >= cp.max(d) for i in range(N-1)]
    hypr = [Ah @ r <= bh + (1 - zh)*bigM, 
            m <= bigM*z, 
            m >= z, 
            Ao @ z.T <= bo.T,
            cp.sum(z) >= N_min]
    
    constraints = ltrig + dist + hypr
    
    if PROB_TYPE == "Power":
        objective = cp.Maximize(cp.min(cp.log(area) + cp.log(irrd)))
    elif PROB_TYPE == "Area":
        objective = cp.Minimize(cp.sum(CP*m + CR*z))
        apwr = [cp.log(area) + cp.log(irrd) >= np.log(power_sim)]
        constraints += apwr

    # Compile and solve
    prob = cp.Problem(objective, constraints)
    writing_time = time.time() - time_start
    
    prob.solve(solver=cp.MOSEK)
    elapsed_time.append(time.time() - time_start)
    
    # Summary
    if prob.status == "optimal":
        print(f"({i+1})", end=" ")
        if (PROB_TYPE == "Power" and prob.value > optim["value"]) or \
           (PROB_TYPE == "Area" and prob.value < optim["value"]):
            optim["value"] = prob.value
            optim["var"]   = prob.var_dict
            optim["azmth"] = curr_arr_azmth
            optim["area"]  = area.value
            optim["power"] = power
            optim["N"] = N
            print("(New)", end=" ")
        print(f"Obj.: {objective.value:.2f}, gamma: {np.rad2deg(curr_arr_azmth):.1f} [deg], " 
              f"Time: (total: {elapsed_time[-1]:.2f}, writing: {writing_time:.2f}) [sec]")
    else:
        print(f"({i+1}) Infeasible.")
        
print(f"Total time: {sum(elapsed_time):.2f} [sec]")

In [ ]:
# Checking variables
var = optim["var"]

summary = f"Tilt (sin): {var['u'].value[0,0]:.2f} \n" + \
          f"Tilt (cos): {var['u'].value[0,1]:.2f} \n" + \
          f"Arrays tilt: {np.rad2deg(np.arctan(var['u'].value[0,0] / var['u'].value[0,1])):.2f} [deg] \n" + \
          f"Trigonometric equality: {cp.sum_squares(var['u']).value:.2f}"
        
print(summary)

# Save variables
with open(FILEPATH_TXT + "layout.txt", "w") as file:
    file.write(summary)

In [ ]:
# Plotting layout (PYOMO)
rects = np.empty((N_max, 4 ,2))
for i in ml.N:
    for c in ml.C:
        rects[i-1,c-1,:] = ml.z[i]*np.array([ml.x[i,c].value, ml.y[i,c].value])

patches = [Polygon(poly, **poly_plot_params)] 
patches += [Polygon(rects[i,:,:], **rect_plot_params) for i in range(N_max)]

fig, ax = plt.subplots(figsize=(FIGSIZE_COL_WIDTH,FIGSIZE_COL_WIDTH))
p = PatchCollection(patches, match_original=True)
ax.add_collection(p)
ax.set_aspect('equal')
ax.axis("off")
ax.autoscale_view()

#plt.savefig(FILEPATH_FIG + f"layout_{PROB_TYPE}.pdf", bbox_inches='tight')
plt.show(block=False)

In [ ]:
# Plotting layout (CVXPY)
rp = cp.bmat([[var["x"], var["x"] + var["m"]*W, var["x"] + var["m"]*W, var["x"]],
              [var["y"], var["y"], var["y"] + var["u"][0,1]*H, var["y"] + var["u"][0,1]*H]])
rp = cp.multiply(rp, cp.kron(np.ones([2,4]), var["z"]))

rpv = rotM(-optim["azmth"]) @ rp.value
rpv = np.reshape(rpv.T, [optim["N"],4,2], order="F")

patches = [Polygon(poly, **poly_plot_params)] 
patches += [Polygon(rpv[i,:,:], **rect_plot_params) for i in range(N)]

fig, ax = plt.subplots(figsize=(FIGSIZE_COL_WIDTH,FIGSIZE_COL_WIDTH))
p = PatchCollection(patches, match_original=True)
ax.add_collection(p)
ax.set_aspect('equal')
ax.axis("off")
ax.autoscale_view()

#plt.savefig(FILEPATH_FIG + f"layout_{PROB_TYPE}.pdf", bbox_inches='tight')
plt.show(block=False)

In [ ]:
# u,v approximation
Q = 3 # number of divisions on axis

# u1u2 surface
u = np.linspace(0,1,20)
u1,u2 = np.meshgrid(u,u)
u1u2 = u1*u2

# Pointwise u1u2 
u_d = np.linspace(0,1,Q)
u1_d,u2_d = np.meshgrid(u_d,u_d)
u1u2_d = u1_d*u2_d

# Create polygons
polygons = []
for i in range(Q-1):
    for j in range(Q-1):
        x = u1_d[[i, i, i+1],[j, j+1, j]]
        y = u2_d[[i, i, i+1],[j, j+1, j]]
        z = u1u2_d[[i, i, i+1],[j, j+1, j]]
        polygons.append(np.vstack([x, y, z]).T)
        
for i in range(1,Q):
    for j in range(1,Q):
        x = u1_d[[i, i, i-1],[j, j-1, j]]
        y = u2_d[[i, i, i-1],[j, j-1, j]]
        z = u1u2_d[[i, i, i-1],[j, j-1, j]]
        polygons.append(np.vstack([x, y, z]).T)

fig, ax = plt.subplots(
#     figsize=(FIGSIZE_COL_WIDTH,FIGSIZE_COL_WIDTH),
    subplot_kw={"projection": "3d"}
)
# No gray background
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# No gray edges
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')

# Plotting 
ax.plot_surface(u1, u2, u1u2, alpha=0.9)
ax.plot3D(u, u, u**2, color="k")
ax.plot3D(u[::-1], u, u*u[::-1], color="k")
ax.add_collection3d(
    Poly3DCollection(polygons, edgecolor="tab:orange", facecolor=None, alpha=0.5, linewidth=1.2)
)
ax.scatter(u1_d.ravel(), u2_d.ravel(), u1u2_d.ravel(), color="tab:orange", s=5)
ax.view_init(elev=4, azim=-110)
plt.xticks(np.linspace(0,1,Q))
plt.yticks(np.linspace(0,1,Q))
plt.xlabel("")
ax.grid(False)

plt.show(block=False)
